In [1]:
import numpy as np

In [2]:
def fitness_val(der_ale,A,zWT,O):
    z_i = zWT + np.matmul(A,der_ale)
    z_sq = np.square(z_i-O)
    w = np.ravel(np.exp(-np.sum(z_sq,axis=0))) #np.ravel to flatten matrix into a 1D array
    #log_w = np.log(w)
    return w

In [3]:
# chance of choosing parent proportional to their fitness
def hybridisation_proportional_fitness(P1,P2,N,A,zWT,O,seedValue):
    rng = np.random.default_rng(seedValue)
    G = 1 #number of generations
    D = P1.shape[0]
    P1 = np.matrix(P1); P2 = np.matrix(P2)
    P1_fit = fitness_val(P1,A,zWT,O)
    P1_choice = np.random.choice(range(N),N,replace=True,p=(P1_fit)/sum(P1_fit))
    S1 = np.zeros([D,1])
    for i in P1_choice:
        s = P1[:,i]
        S1 = np.concatenate((S1,s),axis=1)
    S1 = np.delete(S1,0,1)
    P2_fit = fitness_val(P2,A,zWT,O)
    P2_choice = np.random.choice(range(N),N,replace=True,p=(P2_fit)/sum(P2_fit))
    # sample from the new population with replacement
    S2 = np.zeros([D,1])
    for i in P2_choice:
        s = P2[:,i]
        S2 = np.concatenate((S2,s),axis=1)
    S2 = np.delete(S2,0,1)

    np.place(S1,S1 == 1,rng.binomial(1,0.5,[D,N])*2)
    np.place(S2,S2 == 1,rng.binomial(1,0.5,[D,N])*2)
    F = np.matrix((S1 + S2)/2)

    while 1 in F:
        # sample two parental populations from the new population with replacement. Probability proportional to fitness
        F_fit_1 = fitness_val(F,A,zWT,O)
        P1_choice = np.random.choice(range(N),N,replace=True,p=(F_fit_1)/sum(F_fit_1))
        S1 = np.zeros([D,1])
        for i in P1_choice:
            s = F[:,i]
            S1 = np.concatenate((S1,s),axis=1)
        S1 = np.delete(S1,0,1)
        F_fit_2 = fitness_val(F,A,zWT,O)
        P2_choice = np.random.choice(range(N),N,replace=True,p=(F_fit_2)/sum(F_fit_2))
        # sample from the new population with replacement
        S2 = np.zeros([D,1])
        for i in P2_choice:
            s = F[:,i]
            S2 = np.concatenate((S2,s),axis=1)
        S2 = np.delete(S2,0,1)
        # a new hybridisation
        np.place(S1,S1 == 1,rng.binomial(1,0.5,[D,N])*2)
        np.place(S2,S2 == 1,rng.binomial(1,0.5,[D,N])*2)
        F = (S1 + S2)/2
        G += 1
        #print(f"F {G}")
        # print(F)
    H = np.mean(F)/2
    return F, G, H

# Power of selection

To investigate the effect of selection proportional to fitness, we simulate hybridisation using P1 as a population with individuals that have genotype that gives the best fitness (i.e. $w = 1$).

The extent of selection is measured by the selection coefficient $s$, which is defined as
$$ \frac{w'}{w} - 1 \equiv s$$
where $w$ and $w'$ are the fitness of an individual before and after a change in genotype.

$s$ can be any positive or negative number. When the change in genotype reduces the fitness of the individual, i.e. $w' < w$, $s$ will be negative, and $\textit{vice versa}$.

The above formula can be rearranged as
$$s + 1 = \frac{w'}{w}$$
so
$$\ln(s+1) = \lnw' - \lnw$$

Since $ln(1+s) \approx s$ when $s$ is small, $s \approx \ln{w'} - \ln{w}$ when $s$ is small. In this case where $w=1$, $\ln{w}=0$, so
$$s \approx \ln{w'}$$

### known:
There are two hybridising populations, $P1$ and $P2$.

$P1$ and $P2$ differ genetically at $D$ variable loci, each of which contains two possible alleles, the reference allele and the derived allele. At each locus, an individual can carry 0,1,or 2 copies of the derived allele, so the genotype of any individual can be represented by a vector of length $D$. The genotype at locus $j$, $g_{j}$ ($j=1,2,...,D$), can contain value 0, 1 or 2. The genotype of a population is therefore represented by a $D$ by $N$ matrix called the $G$ matrix.

#### from genotype to phenotype


(Assume Fisher's geometric model)
Assume that there is no interaction between the two alleles of the same locus in a diploid genome. A trait that is associated with the locus will have a value of the trait value of the reference genotype plus the additional effect brought by any number of derived alleles (i.e. if the phenotypic effect of a homozygous reference allele on a particular trait is $z_{WT,i}$, and the phenotypic effect of one derived allele one the same trait is $a_{i}$, then a genotype of 0 will have trait value $z_{WT,i}$, a genotype of 1 will have $z_{WT,i} + a_{i}$ and a genotype of 2 will have $z_{WT,i} + 2a_{i}$.

Each genetic locus affects the phenotype of an individual at one or more traits. The total number of traits that could be affected by D variable loci is n. The phenotypic effect of all D loci on n traits can be represented by an n by D matrix called the $A$ matrix, with the effect of locus $j (j = 1,2,...,D)$ on trait $i (i = 1,2,...,n)$ represented as $a_{ij}$.

The trait value of a reference genotype (with no derived alleles) is denoted by z_{WT}.

For an arbitrary genotype, the value of trait i is given by
$$
z_{i} = z_{WT,i} + \sum_{j=1}^{D} g_j\,a_{ij}
$$

#### from phenotype to fitness
The fitness of a genotype, $w$, follows directly from all of its traits values. The optimal value of each trait is $o_{i}$ for trait $i$. The overall fitness of an individual is calculated by
$$
w = exp(- \sum_{i=1}^{n}(z_{i} - o_{i})^2)
$$
Note that, if an individual has an optimum value at all traits, it will have an overall fitness of $e^{-0} = 1$. If an individual has a trait value that deviates from the optimum trait value in either direction (i.e. too large or oo small), its fitness will be less than 1. The minimum fitness value is zero.

### what do we start with:
All individuals from both populations are homozygous in their variable loci. At all of their loci, the $P1$ population contains the reference allele, and the $P2$ population contains the derived allele. Therefore, the G matrix for P1 is a matrix of 0s, and the G matrix for P2 is a matrix of 2s.

## Epistasis in this model
while the phenotypic effects of different gene loci are purely additive, their effect on fitness can have interation between each other.

When each trait is affected by only one locus, the sum of log fitness of all individual loci is equal to the sum of log fitness of all individual trait. When there are more than one gene locus affecting the same trait, there exists an additional fitness effect. This is because while individual log fitness effects of one particular locus is the sum of squares of the phenotypic effects it has on all traits, total fitness effects is calculated as the sum of fitness effects of all the traits, each is calculated by summing the effects from all loci and then squaring.

(matrix A is D by n)

Note that the total fitness value.

## Case 1: Additive effects only

In a simple case, each locus only affects one trait, and each trait is affected by only one locus. Matrix A can be represented by an n by n matrix with non-zero values in the diagonal only:

$$
\left(\begin{array}{cc}
a_{1} & 0 &0 & \dots & 0  \\
0 & a_{2} & 0 & \dots & 0 \\
0 & 0  & \ddots & \dots & \vdots  \\
\vdots & \vdots & \vdots & \ddots & 0 \\
0 & 0 & \dots & 0 & a_{n}
\end{array}\right)

$$

For simplicity, all the non-zero values are set as z.

In [4]:
D = 5; N = 10; n = 5
# Create an A matrix of dimension n*D, n = D. Each gene locus has a phenotypic effect of z on their respective trait.
z = 0.01
A= np.diagflat(np.full([1,D],z))

P1 = np.zeros([D,N]); P2 = np.full([D,N],2)
# zWT and O both zeros
zWT = np.zeros([n,N])
O = np.zeros([n,N])

In [5]:
w1 = fitness_val(P1,A,zWT,O)
w2 = fitness_val(P2,A,zWT,O)
print(w1)
print(w2)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.998002 0.998002 0.998002 0.998002 0.998002 0.998002 0.998002 0.998002
 0.998002 0.998002]


In [6]:
w = hybridisation_proportional_fitness(P1,P2,N,A,zWT,O,None)
print(w)

(matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]]), 53, 0.4)


In [7]:
s = np.log(w2[0])
print(f's is equal to {s}')

s is equal to -0.001999999999999988


Selection coefficient s is approximated as $ln(w2)$.

Note that when $N|s| > 1$, selection is effective.

In this case where N = 10, s needs to be greater than 0.1 to be effective.

In [ ]:
# Change value of z. Show generation time decreases as s increases, and final population are more likely to be all 0s. Use N = 20.


# Maximum epistasis

Matrix A has one trait affected by all loci.